# Preamble
This Jupyter notebook introduces the use of Python packages for geoscripting as part of a "pipeline" workflow suitable for use with Google Earth Engine Python API. This is meant to serve as an example, particularly as it is still a work in progress. 

# Steps
#### 1) Find one dam site and determine analysis extent
1a) Download Grand Database: http://sedac.ciesin.columbia.edu/data/set/grand-v1-dams-rev01/data-download
and select candidate sites (points):
- TODAY: "Ham Thuan" in Vietnam - 300 MW - construction between 1997 and 2001
- "Porce 2" Dam in Colombia - 660 MW - constructed between 2004 and 2011

1b) Set the study area by creating a feature of the buffered area around dam site 

#### 2) Acquire and process Landsat data 
2a) Gather two Landsat 5 images for comparison

#### 3) Calculate NDWI
3a) Calculate NDWI 
3b) Use NDWI at threshold to mask water

#### 4) Calculate NDVI 
4a) Calculate NDVI of water layer

## To try on your own:
#### 1) Calculate a normalized Built-Up Area Index  [NDBI=(TM5−TM4)/(TM5+TM4)]
http://www.tandfonline.com/doi/abs/10.1080/01431160304987
#### 2) Apply a cloud mask (need to use TOA and merge with SR)
see this script: https://code.earthengine.google.com/15d0e461d37a100d051e50443fb9f6d8
#### 3) Apply steps 2-4 to an "Image Collection" using the .map method

### Other: Plot NDVI histogram?

### Other: Classify images (water, vegetation, built-up area)
Example of CART classifier


## Import Juypter notebook extensions

In [1]:
import notebook
notebook.nbextensions.check_nbextension('usability/codefolding', user=True) 
notebook.nbextensions.check_nbextension('usability/codefolding/main.js', user=True)
E = notebook.nbextensions.EnableNBExtensionApp()
E.enable_nbextension('usability/codefolding/main')

## Install extensions using the following instructions:
# https://github.com/ipython-contrib/IPython-notebook-extensions#installation
## if you want to use the GUI for managing extensions:
## https://github.com/ipython-contrib/IPython-notebook-extensions/wiki/Config-Extension
## manage them using http://localhost:8888/nbextensions

## Import packages

In [2]:
## for reading and writing data (wrapper for ogr/gdal that is more efficient and pythonic)
import fiona
from fiona import collection
## https://github.com/Toblerity/Fiona
## http://toblerity.org/fiona/manual.html#format-drivers-crs-bounds-and-schema

## for working with vector data 
from shapely.geometry import mapping, shape ## for geoprocessing vector data
# fiona.ogrext.get_gdal_release_name()

import pprint
import numpy
import matplotlib

## Import point shapefile and view attributes

In [4]:
damFile = '/Users/kschocz/jupyterNotebooks/data/GRanD_dams_v1_1.shp'
with fiona.collection(damFile, 'r') as layer:
    ## The structure or design of a database or database object--it gives you the geometry and properties/fields
    pprint.pprint(layer.schema)
    print ""
    ## get number of records/rows
    print "number of elements :", len(layer)
    print ""
    ## see driver used to open the file
    print layer.driver
    print ""
    ## get coordinate reference system(CRS)
    print layer.crs
    print ""

{'geometry': 'Point',
 'properties': OrderedDict([(u'GRAND_ID', 'int:8'), (u'RES_NAME', 'str:50'), (u'DAM_NAME', 'str:50'), (u'ALT_NAME', 'str:50'), (u'RIVER', 'str:50'), (u'ALT_RIVER', 'str:50'), (u'MAIN_BASIN', 'str:50'), (u'SUB_BASIN', 'str:50'), (u'NEAR_CITY', 'str:50'), (u'ALT_CITY', 'str:50'), (u'ADMIN_UNIT', 'str:50'), (u'SEC_ADMIN', 'str:50'), (u'COUNTRY', 'str:50'), (u'SEC_CNTRY', 'str:50'), (u'YEAR', 'int:4'), (u'ALT_YEAR', 'int:4'), (u'DAM_HGT_M', 'int:4'), (u'ALT_HGT_M', 'int:4'), (u'DAM_LEN_M', 'int:8'), (u'ALT_LEN_M', 'int:8'), (u'AREA_SKM', 'float:10.1'), (u'AREA_POLY', 'float:10.1'), (u'AREA_REP', 'float:10.1'), (u'AREA_MAX', 'float:10.1'), (u'AREA_MIN', 'float:10.1'), (u'CAP_MCM', 'float:10.1'), (u'CAP_MAX', 'float:10.1'), (u'CAP_REP', 'float:10.1'), (u'CAP_MIN', 'float:10.1'), (u'DEPTH_M', 'float:8.1'), (u'DIS_AVG_LS', 'int:10'), (u'DOR_PC', 'float:8.1'), (u'ELEV_MASL', 'int:4'), (u'CATCH_SKM', 'int:8'), (u'CATCH_REP', 'int:8'), (u'DATA_INFO', 'str:50'), (u'USE_IRRI',

## Create buffer feature

In [7]:
# Lets see how to set a working dir! 
#print '/Users/kschocz/jupyterNotebooks'
curDir = '/Users/kschocz/jupyterNotebooks'
print curDir
print curDir + '/data'

/Users/kschocz/jupyterNotebooks
/Users/kschocz/jupyterNotebooks/data


In [8]:
## Buffer file name
bufferFile = curDir + '/hamThuan_buffer.shp'

## Create buffer around selected point record
## and save to disk using above file name
## Fiona objects are modeled on the GeoJSON format and don't have any spatial methods 
## of their own, so if you want to do anything fancy with them you will probably need something
## like Shapely
with collection(damFile, "r") as input:
    # schema = input.schema.copy()
    schema = { 
        'geometry': 'Polygon', 
        'properties': { 
            'DAM_NAME': 'str' 
        } 
    }
    with collection(bufferFile, "w", "ESRI Shapefile", schema) as output:
        hamThuan = filter(lambda f: f['properties']['DAM_NAME'] == 'Ham Thuan 1', input)
        for point in hamThuan:
            output.write({
                'properties': {
                        'DAM_NAME': point['properties']['DAM_NAME']
                    },
                'geometry': mapping(shape(point['geometry']).buffer(0.15))
            })  
# with fiona.open() as dams:
#    hamThuan = filter(lambda f: f['properties']['DAM_NAME']=='Ham Thuan 1', dams)
## Check 
pprint.pprint(hamThuan)

# geomet = dams.shapeRecords() #will store the geometry separately
#first = geomet[0] #will extract the first polygon to a new object
#first.shape.points #will show you the points of the polygon
#first.record #will show you the attributes

[{'geometry': {'coordinates': (107.93374999999929, 11.325416666662619),
               'type': 'Point'},
  'id': '5800',
  'properties': OrderedDict([(u'GRAND_ID', 5801), (u'RES_NAME', None), (u'DAM_NAME', u'Ham Thuan 1'), (u'ALT_NAME', None), (u'RIVER', u'La Nga'), (u'ALT_RIVER', None), (u'MAIN_BASIN', u'Mekong'), (u'SUB_BASIN', None), (u'NEAR_CITY', u'Bao Loc'), (u'ALT_CITY', None), (u'ADMIN_UNIT', None), (u'SEC_ADMIN', None), (u'COUNTRY', u'Vietnam'), (u'SEC_CNTRY', None), (u'YEAR', 2001), (u'ALT_YEAR', -99), (u'DAM_HGT_M', 93), (u'ALT_HGT_M', 94), (u'DAM_LEN_M', 586), (u'ALT_LEN_M', -99), (u'AREA_SKM', 28.6), (u'AREA_POLY', 28.6), (u'AREA_REP', 25.2), (u'AREA_MAX', -99.0), (u'AREA_MIN', -99.0), (u'CAP_MCM', 695.0), (u'CAP_MAX', -99.0), (u'CAP_REP', 695.0), (u'CAP_MIN', -99.0), (u'DEPTH_M', 24.3), (u'DIS_AVG_LS', 26029), (u'DOR_PC', 84.7), (u'ELEV_MASL', 567), (u'CATCH_SKM', 1290), (u'CATCH_REP', -99), (u'DATA_INFO', None), (u'USE_IRRI', None), (u'USE_ELEC', u'Main'), (u'USE_SUPP', 

In [5]:
## View all methods for a fiona collection
# help(input)

In [9]:
## Open the saved BufferFile (shp) again and convert it to a fiona feature collection (effectively a geojson)
# Fiona’s Collection is like a Python file, but is iterable for records rather than lines.
# Python has a built-in function, open, for opening a file on disk. open returns a "file object," 
# which has methods and attributes for getting information about and manipulating the opened file.
bufferPoly = fiona.open(bufferFile)

pprint.pprint(bufferPoly)
# next(bufferPoly) ## iterates records
# len(list(bufferPoly))

<open Collection '/Users/kschocz/jupyterNotebooks/hamThuan_buffer.shp:hamThuan_buffer', mode 'r' at 0x10c7fbed0>


# The Google Maps Widget (module from Google)

In [10]:
## import and initialize earth engine API
import ee
ee.Initialize()
print ee.__version__
from IPython.display import display

0.1.77


In [11]:
%%javascript

require(['widgets/js/widget', 'widgets/js/manager', 'jquery', 'underscore'], function(widget, manager, $, _) {
    /**
     * A simple model to represent a layer on the map.
     *
     * @constructor
     */
    var Layer = Backbone.Model.extend({
        defaults: function() {
            return {
                config: {},
                type: undefined,
                visible: true
            };
        }
    });



    /**
     * A collection of layers.
     *
     * @constructor
     */
    var LayerCollection = Backbone.Collection.extend({
        model: Layer
    });



    /**
     * Override of the main widget model to intercept messages from Python
     * update Javascript state correctly.
     *
     * @constructor
     */
    var GoogleMapsModel = widget.WidgetModel.extend({
        /*
        defaults: function() {
            return {
                layers: new LayerCollection()
            };
        },
        */

        /** @override */
        initialize: function() {
            this.listenTo(
                this, 'msg:custom', _.bind(this.handleMessage, this));
            // this.set('layers', new LayerCollection());
        },

        /**
         * Handle a message from Python.
         *
         * @param {!Object} payload Payload of the message.
         */
        handleMessage: function(payload) {
            if (!this.get('layers')) {
                this.set('layers', new LayerCollection());
            }
            switch(payload.action) {
                case 'addLayer':
                    this.get('layers').add({
                        config: payload.config,
                        type: payload.type
                    });
                    break;
                case 'removeLayer':
                    console.error('removeLayer not implemented');
                    break;
            }
        }
    });

    // Register the model.
    manager.WidgetManager.register_widget_model(
        'GoogleMapsModel', GoogleMapsModel);



    /**
     * A Google Maps API widget.
     *
     * @constructor
     */
    var GoogleMapsView = widget.DOMWidgetView.extend({
        /**
         * Load the Maps API JS if needed, also prepare a deferred in case any
         * map methods are called before the map is ready.
         */
        initialize: function() {
            // Deferred to track for when the map is ready.
            this.mapsReadyDeferred = $.Deferred();

            // Dynamically adding Google Maps API JS here. Using a deferred to
            // track its load status as require returns as soon as the first
            // script loads and the Maps API triggers more scripts to append
            // which leaves a race condition where require thinks Maps API JS
            // is ready when it is not yet.
            var mapsDeferred = this.mapsDeferred = $.Deferred();
            // Another instance of this view may have already loaded the Maps
            // API JS, do not try to load it twice.
            if (window.google && window.google.maps) {
                mapsDeferred.resolve();
            } else {
                window.googleMapsCallback = function() {
                    mapsDeferred.resolve();
                };
                require(
                    ['http://maps.googleapis.com/maps/api/js?callback=googleMapsCallback'],
                    function() {},
                    function() {});
            }
        },

        /**
         * Render the map.
         */
        render: function() {
            // We must wait until the Maps API JS is ready.
            $.when(this.mapsDeferred.promise()).then(_.bind(function() {
                // Empty the views DOM. There seem to be some weird side
                // effects when you render more than one instance of this view
                // in the notebook. Cleaning the view DOM and deferring the map
                // initialization seems to work around this. It seesm almost if
                // map instances are sharing some DOM somehow.
                this.$el.empty();
                _.defer(_.bind(function() {
                    this.$map = $([
                        '<div style="height: ',
                        this.model.get('height'),
                        '; width: ',
                        this.model.get('width'),
                        ';"></div>'
                    ].join(''));
                    this.$el.append(this.$map);
                    this.map = new google.maps.Map(this.$map.get(0), {
                        center: {
                            lat: this.model.get('lat'),
                            lng: this.model.get('lng')
                        },
                        zoom: this.model.get('zoom')
                    });

                    // Notify the map when the map container changes size via
                    // the exposed properties in the model.
                    this.listenTo(
                        this.model, 'change:height', _.bind(function() {
                            this.$map.height(this.model.get('height'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));
                    this.listenTo(
                        this.model, 'change:width', _.bind(function() {
                            this.$map.width(this.model.get('width'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));

                    // Bind a change in the position of the map to the model.
                    google.maps.event.addListener(
                        this.map,
                        'bounds_changed',
                        _.bind(this.syncFromMap, this));

                    // Bind a change in the model (coming from the Python side)
                    // to the location of the map.
                    this.listenTo(
                        this.model,
                        'change:lat',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:lng',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:zoom',
                        _.bind(this.syncFromModel, this));

                    // Render the initial set of layers.
                    if (!this.model.get('layers')) {
                        this.model.set('layers', new LayerCollection());
                    }
                    this.model.get('layers').each(this.buildLayer, this);

                    // Bind to changes in the layers of the model to stay in
                    // sync.
                    this.listenTo(
                        this.model.get('layers'),
                        'add',
                        _.bind(this.buildLayer, this));
                    this.listenTo(
                        this.model.get('layers'),
                        'remove',
                        function() {
                            console.error('removeLayer not implemented');
                        });

                    // Even though a google.maps.Map instance should be ready
                    // immediately, it is not. This delay lets the stack clear
                    // and initial map bounds to be set.
                    _.delay(_.bind(function() {
                        this.mapsReadyDeferred.resolve();
                    }, this), 500);
                }, this));
            }, this));
        },

        /**
         * Sync the values from the map into the model.
         */
        syncFromMap: function() {
            this.model.set({
                lat: this.map.getCenter().lat(),
                lng: this.map.getCenter().lng(),
                zoom: this.map.getZoom()
            });
            // This is needed for the model to update the equivalent properties
            // on the Python instance of this view.
            this.model.save_changes();
        },

        /**
         * Move the map to match the values from the model.
         */
        syncFromModel: function() {
            this.map.setCenter(new google.maps.LatLng(
                this.model.get('lat'),
                this.model.get('lng')
            ));
            this.map.setZoom(this.model.get('zoom'));
        },

        /**
         * Add a layer to the map based on its model.
         *
         * @param {!Layer} layer The layer to add.
         */
        buildLayer: function(layer) {
            switch(layer.get('type')) {
                case 'geojsondata':
                    this.addGeoJsonLayer(layer.get('config').data);
                    break;
                case 'geojsonurl':
                    this.loadGeoJsonLayer(layer.get('config').url);
                    break;
                case 'kmlurl':
                    this.loadKmlLayer(layer.get('config').url);
                    break;
                case 'earthengine':
                    this.addEarthEngineLayer(
                        layer.get('config').mapid, layer.get('config').token);
                    break;
            }
        },

        /**
         * Adds GeoJSON to the map.
         *
         * @param {!Object} data A GeoJSON object.
         */
        addGeoJsonLayer: function(data) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                this.map.data.addGeoJson(data);
            }, this));
        },

        /**
         * Adds a URL location of GeoJSON to the map.
         *
         * @param {string} url The URL of the GeoJSON file to load.
         */
        loadGeoJsonLayer: function(url) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                this.map.data.loadGeoJson(url);
            }, this));
        },

        /**
         * Add a KML layer to the map.
         *
         * @param {string} url The URL of the KML file to load.
         */
        loadKmlLayer: function(url) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                new google.maps.KmlLayer({
                    url: url,
                    map: this.map
                });
            }, this));
        },

        /**
         * Add an Earth Engine layer to the map.
         *
         * @param {string} mapid The id of the Earth Engine layer.
         * @param {string} token The OAuth token to authenticate with.
         */
        addEarthEngineLayer: function(mapid, token) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                var eeMapOptions = {
                    getTileUrl: function(tile, zoom) {
                        var url = [
                            'https://earthengine.googleapis.com/map',
                            mapid,
                            zoom,
                            tile.x,
                            tile.y
                        ].join('/');
                        url += '?token=' + token;
                        return url;
                    },
                    tileSize: new window.google.maps.Size(256, 256),
                    opacity: 1.0,
                };

                // Create the overlay map type.
                var mapType = new window.google.maps.ImageMapType(eeMapOptions);

                // Overlay the Earth Engine generated layer.
                this.map.overlayMapTypes.push(mapType);
            }, this));
        }
    });

    // Register the view.
    manager.WidgetManager.register_widget_view(
        'GoogleMapsView', GoogleMapsView);
});

<IPython.core.display.Javascript object>

# Configure Python half of widget (module from Google)

In [13]:
from ipywidgets import widgets
import traitlets

class GoogleMapsView(widgets.DOMWidget):
    """Google Maps API widget."""
    _model_name = traitlets.Unicode('GoogleMapsModel', sync=True)
    _view_name = traitlets.Unicode('GoogleMapsView', sync=True)
    lat = traitlets.CFloat(0, sync=True)
    lng = traitlets.CFloat(0, sync=True)
    zoom = traitlets.CInt(2, sync=True)

    height = traitlets.CUnicode('300px', sync=True)
    width = traitlets.CUnicode('400px', sync=True)

    layers = traitlets.List([], sync=False)

    def addGeoJsonLayer(self, data):
        """Adds a dictionary of GeoJSON to the map.

        NOTE: It is likely if you are using a third party GeoJSON library you
        will have to first serialize the data into a simple dictionary before
        passing the data to this method.

        Args:
            data: A simple python dictionary of GeoJSON data.
        """
        self.send({
            'action': 'addLayer',
            'type': 'geojsondata',
            'config': {'data': data}
        })

    def loadGeoJsonLayer(self, url):
        """Adds a URL location of GeoJSON to the map.

        Args:
            url: The URL of the GeoJSON file.
        """
        self.send({
            'action': 'addLayer',
            'type': 'geojsonurl',
            'config': {'url': url}
        })

    def loadKmlLayer(self, url):
        """Adds a KML layer to the map.

        Args:
            url: The URL of the KML file.
        """
        self.send({
            'action': 'addLayer',
            'type': 'kmlurl',
            'config': {'url': url}
        })

    def addEarthEngineLayer(self, image, vis_params):
        """Adds an Earth Engine layer to the map.

        Args:
            image: The ee.Image to display.
            vis_params: Dictionary of visualization parameters.
        """
        mapid = image.getMapId(vis_params)
        self.send({
            'action': 'addLayer',
            'type': 'earthengine',
            'config': {
                'mapid': mapid['mapid'],
                'token': mapid['token']
            }
        })


/anaconda/envs/eepython/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: metadata {'sync': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
/anaconda/envs/eepython/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: metadata {'sync': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
/anaconda/envs/eepython/lib/python2.7/site-packages/ipykernel/__main__.py:8: DeprecationWarning: metadata {'sync': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
/anaconda/envs/eepython/lib/python2.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: metadata {'sync': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
/anaconda/envs/e

# Visualize vector inputs using google maps widget

In [14]:
## Create the base Google maps widget:
widget = GoogleMapsView(lng=107.9337, lat=11.3254, zoom= 10, height='400px', width='800px')

## Create map widget layers to link.
map1a = GoogleMapsView(lng=widget.lng, lat = widget.lat, zoom = widget.zoom, height=widget.height, width=widget.width)
map1b = GoogleMapsView(height=map1a.height, width=map1a.width)

## Link the map widget layers.
widgets.jslink((map1a, 'lat'), (map1b, 'lat'))
widgets.jslink((map1a, 'lng'), (map1b, 'lng'))
widgets.jslink((map1a, 'zoom'), (map1b, 'zoom'))

## The following loads a GeoJSON layer directly from a fiona object to the widget
## Just remember that a fiona object is a list of dictionaries (a record is an item in the list), 
## so indicate which dictionary you want to add to display (in this case there is only one point 
## in the list).
map1a.addGeoJsonLayer(hamThuan[0])
map1b.addGeoJsonLayer(bufferPoly[0])

## Display the widget layers
display(map1a)
display(map1b)

example of a geojson for point file:
geojson_map.addGeoJsonLayer({'type': 'Feature',
    'geometry': {
        'type': 'Point',
        'coordinates': [107.93374999999929, 11.325416666662619]
    },
    'properties': {
        'name': 'Dinagat Islands'
    }
})

## Use Earth Engine to access landsat imagery

In [15]:
## Create a reference to the Image collection
## Landsat 5 LEDAPS: LEDAPS/LT5_L1T_SR (Only calculates reflectance for optical bands)
## Landsat 5 TOA: LANDSAT/LT5_L1T_TOA
## L5 Data availability (time) Jan 1, 1984 - May 5, 2012
## Reflectance is a unitless ratio rescaled to 0-10000
## See doc for more info about image collection: https://code.earthengine.google.com/dataset/LEDAPS/LT5_L1T_SR
## Path:124 Row:52

collection = ee.ImageCollection('LEDAPS/LT5_L1T_SR')

## Located the following two relatively cloud-free images on USGS Earth Explorer:
## preDam image: LT51240521996061CLT00 (01-MAR-96)
## postDam Image: LT51240522005053BKT00 (22-FEB-05)
preDam = collection.filterDate('1996-03-01', '1996-03-02').mode()
postDam = collection.filterDate('2005-02-21', '2005-02-23').mode()

## Convert geojson to eeFeature object (the clip method on an eeImage only accepts eeFeatures)
feature_bufferPoly = ee.Feature(bufferPoly[0]);

## Get Info
## print(preDam_clipped.count())
preDam_clipped = preDam.clip(feature_bufferPoly)
postDam_clipped = postDam.clip(feature_bufferPoly)

#######################################################
## ASIDE: for multiple images in an image collection ##
#######################################################

## Clip function using feature_bufferPoly
## Apply a function to every Image in an ImageCollection using imageCollection.map(). 
## The only argument to map() is a function which takes one parameter: an ee.Image.
#def clipCollection(image):
#    return image.clip(feature_bufferPoly)

## "Map" post dam function 
# preDam_clipped = preDam.map(clipCollection)
# postDam_clipped = postDam.map(clipCollection)


### Display clipped images

In [16]:
## 200 - 2000 for surface reflectance values
map1a.addEarthEngineLayer(image=preDam_clipped, vis_params={'min': 200, 'max': 2000, 'bands': 'B3,B2,B1'})
display(map1a)

map1b.addEarthEngineLayer(image=postDam_clipped, vis_params={'min': 200, 'max': 2000, 'bands': 'B3,B2,B1'})
display(map1b)

## Calculate NDWI

In [17]:
## Calculate NDWI
## [NDWI = (Xgreen – Xnir)/(Xgreen + Xnir)], 
## where Xgreen refers to the green band (Landsat 5 band 2) and Xnir refers to the nir band (Landsat 5 band 4)

## simple function to calculate NDWI
def NDWI(image):
    return image.normalizedDifference(['B2','B4'])

## Function that masks image using threshold 
def applyMask(maskImage, image, lte_value):
    mask = maskImage.lte(lte_value)
    return image.updateMask(mask)
    
## Apply NDWI to both images
preDam_NDWI = NDWI(preDam_clipped)
postDam_NDWI = NDWI(postDam_clipped)

## Visualize NDWI
waterWidget1 = GoogleMapsView(lng=107.9337, lat=11.3254, zoom= 10, height='400px', width='800px')
waterWidget1.addEarthEngineLayer(preDam_NDWI, vis_params={min: -0.3, max: 0.3})

waterWidget2 = GoogleMapsView(lng=107.9337, lat=11.3254, zoom= 10, height='400px', width='800px')
waterWidget2.addEarthEngineLayer(postDam_NDWI, vis_params={min: -0.3, max: 0.3})

display(waterWidget1) 
display(waterWidget2)

## Mask the water

In [18]:
## Apply water mask using applyMask function
preDam_land = applyMask(preDam_NDWI, preDam_clipped, -0.1)
postDam_land = applyMask(postDam_NDWI, postDam_clipped, -0.1)

## Visualize masked images
maskedWidget1 = GoogleMapsView(lng=107.9337, lat=11.3254, zoom= 10, height='400px', width='800px')
maskedWidget1.addEarthEngineLayer(preDam_land, vis_params={'min': 200, 'max': 2000, 'bands': 'B3,B2,B1'})

maskedWidget2 = GoogleMapsView(lng=107.9337, lat=11.3254, zoom= 10, height='400px', width='800px')
maskedWidget2.addEarthEngineLayer(postDam_land, vis_params={'min': 200, 'max': 2000, 'bands': 'B3,B2,B1'})

display(maskedWidget1) 
display(maskedWidget2)

## Calculate NDVI

In [19]:
## NDVI = (NIR — VIS)/(NIR + VIS) NIR=B4, RED = B3
## Simple function that calculates NDVI
def NDVI(image):
    return image.normalizedDifference(['B4','B3'])

## Calculate NDVI
preDam_NDVI = NDVI(preDam_land)
postDam_NDVI = NDVI(postDam_land)

## Visualize NDVI images
NDVIWidget1 = GoogleMapsView(lng=107.9337, lat=11.3254, zoom= 10, height='400px', width='800px')
NDVIWidget1.addEarthEngineLayer(preDam_NDVI, vis_params={min: -1, max: 1,})

NDVIWidget2 = GoogleMapsView(lng=107.9337, lat=11.3254, zoom= 10, height='400px', width='800px')
NDVIWidget2.addEarthEngineLayer(postDam_NDVI, vis_params={min: -1, max: 1})

display(NDVIWidget1) 
display(NDVIWidget2)

## Download NDVI of each image for plotting (work in progress)

In [20]:
#preDam_info = preDam_NDVI.getInfo()
#postDam_info = postDam_NDVI.getInfo()

# preDam_hist = postDam_NDVI.reduce(ee.Reducer.histogram());
## print preDam_hist
import json
visparams = {'name': 'preDam_NDVI',
                     #'bands':  [b4,b3,b2], # none of the above work.
                     #'crs': crs,
                     'format': 'tiff',
                     #'scale': 30,
                     #'gain':  0.1, 
                     'region' : json.dumps(bufferPoly[0]["geometry"]["coordinates"]),
                     'filePerBand' : False}
path = preDam_NDVI.getDownloadURL(visparams)

In [28]:
## Download from URL
fileName = '/Users/GraceWu/Desktop/geoscriptingPython_tutorial/preDam_NDVI.zip'
import urllib
preDam_dl = urllib.URLopener()
preDam_dl.retrieve(path,fileName)

('/Users/GraceWu/Desktop/geoscriptingPython_tutorial/preDam_NDVI.zip',
 <httplib.HTTPMessage instance at 0x1103f8950>)

In [29]:
## Open as Numpy Array
import zipfile
import numpy
import gdal
from osgeo.gdalconst import GA_ReadOnly, GDT_Float32

with zipfile.ZipFile(fileName, 'r') as f:
    f.extractall(fileName[:-4])
    
data = gdal.Open(fileName[:-4] + "/preDam_NDVI.tif", GA_ReadOnly)
print data.GetDriver().LongName
print data.RasterCount

GeoTIFF
1


## Extra: Classify each landsat image

In [32]:
trainingData = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[107.92797088623047, 11.363924114853964],
                  [107.92522430419922, 11.353152965512512],
                  [107.92762756347656, 11.343727876301832],
                  [107.93346405029297, 11.353826174261734]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.92659759521484, 11.331946077410572],
                  [107.92316436767578, 11.32655995046335],
                  [107.92831420898438, 11.322520288693578],
                  [107.93071746826172, 11.3275698569927]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.89552688598633, 11.319574666032517],
                  [107.8938102722168, 11.316376527106245],
                  [107.89578437805176, 11.313262515404313],
                  [107.89775848388672, 11.317133984291342]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.94281959533691, 11.402377161263042],
                  [107.94110298156738, 11.397917884039359],
                  [107.94290542602539, 11.39303784010575],
                  [107.94599533081055, 11.396150981255783],
                  [107.94754028320312, 11.401283382761246]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.91921615600586, 11.37797648688004],
                  [107.91801452636719, 11.376377748962886],
                  [107.9216194152832, 11.371076606389185],
                  [107.92264938354492, 11.37217050103064]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.93998718261719, 11.39261714274123],
                  [107.93801307678223, 11.390429506412517],
                  [107.93835639953613, 11.386559031687037],
                  [107.94290542602539, 11.387484584781939]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.91664123535156, 11.357781243584228],
                  [107.91423797607422, 11.355761640568089],
                  [107.91586875915527, 11.35458353220608],
                  [107.91818618774414, 11.35702389412877]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.91166305541992, 11.321426203815808],
                  [107.9102897644043, 11.31831224704035],
                  [107.91106224060059, 11.316965660645613],
                  [107.91406631469727, 11.318396408479707]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.91260719299316, 11.33017876568831],
                  [107.90951728820801, 11.327738174400901],
                  [107.91071891784668, 11.324540126708358],
                  [107.91509628295898, 11.32748569825143]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.90642738342285, 11.33783703757724],
                  [107.9036808013916, 11.3359856060366],
                  [107.90505409240723, 11.333124279156388],
                  [107.9080581665039, 11.334975729231875]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.90900230407715, 11.392785421761738],
                  [107.90857315063477, 11.392112305082184],
                  [107.9091739654541, 11.387905289738713],
                  [107.91003227233887, 11.388410134865545]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.92410850524902, 11.37158148136102],
                  [107.92170524597168, 11.369225390504718],
                  [107.9227352142334, 11.361315514453041],
                  [107.92779922485352, 11.365691217693298]]]),
            {
              "classvalue": 0,
              "classname": "water",
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.96633720397949, 11.325550040368483],
                  [107.96470642089844, 11.322688609072724],
                  [107.96719551086426, 11.319069699103759],
                  [107.97019958496094, 11.320752918735055]]]),
            {
              "classvalue": 1,
              "classname": "land",
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.8696060180664, 11.333965848859103],
                  [107.86582946777344, 11.323530209483287],
                  [107.87235260009766, 11.31747063128591],
                  [107.87372589111328, 11.324876764991187]]]),
            {
              "classvalue": 1,
              "classname": "land",
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.94264793395996, 11.262760308084388],
                  [107.9377555847168, 11.259561536010617],
                  [107.94290542602539, 11.255689290668451],
                  [107.9472827911377, 11.259814071940804]]]),
            {
              "classvalue": 1,
              "classname": "land",
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.88728713989258, 11.277406862658774],
                  [107.88265228271484, 11.275639214634065],
                  [107.88351058959961, 11.270925433415794],
                  [107.88883209228516, 11.27109378407606]]]),
            {
              "classvalue": 1,
              "classname": "land",
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.84523010253906, 11.28843337381658],
                  [107.83827781677246, 11.286076597849158],
                  [107.84042358398438, 11.28127881556843],
                  [107.84651756286621, 11.28127881556843]]]),
            {
              "classvalue": 1,
              "classname": "land",
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.83956527709961, 11.310569288788749],
                  [107.83381462097168, 11.305266924897342],
                  [107.83553123474121, 11.302321124788058],
                  [107.84308433532715, 11.306276906536521]]]),
            {
              "classvalue": 1,
              "classname": "land",
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[107.89587020874023, 11.33733210289208],
                  [107.89320945739746, 11.335228198767588],
                  [107.89535522460938, 11.329589659353513],
                  [107.89887428283691, 11.33059955518439]]]),
            {
              "classvalue": 1,
              "classname": "land",
              "system:index": "18"
            })])

In [39]:
## CART CLASSIFIER

## Use these bands for prediction
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']

## Get the values for all pixels in each polygon in the training.
training = postDam_clipped.sampleRegions(
  ## Get the sample from the polygons FeatureCollection.
  collection = trainingData,
  ## Keep this list of properties from the polygons.
  properties= ['classvalue'],
  ## Set the scale to get Landsat pixels in the polygons.
  scale= 30
)

## Train a CART classifier with default parameters.
trained = ee.Classifier.cart().train(training, 'classvalue', bands)

## Classify the image with the same bands used for training.
classified = postDam_clipped.select(bands).classify(trained)

## Create a palette to display the classes.
palette = ','.join(['0000FF', "6a2325"])  ## creates a string of palette hexadecimal values

## Display the classification result

widget_postDam = GoogleMapsView(lng=107.9337, lat=11.3254, zoom= 10, height='240px', width='800px')
widget_classified = GoogleMapsView(lng=107.9337, lat=11.3254, zoom= 10, height='240px', width='800px')
widgets.jslink((widget_postDam, 'lat'), (widget_classified, 'lat'))
widgets.jslink((widget_postDam, 'lng'), (widget_classified, 'lng'))
widgets.jslink((widget_postDam, 'zoom'), (widget_classified, 'zoom'))

widget_postDam.addEarthEngineLayer(postDam_land, vis_params={'min': 200, 'max': 2000, 'bands': 'B3,B2,B1'})
widget_classified.addEarthEngineLayer(classified, vis_params={"min": 0, "max": 1, "palette": palette})

display(widget_postDam)
display(widget_classified)

## Extra: Apply cloud mask and filter image collection to least-cloudy images

In [124]:
## Function to add a cloud score band.  It is automatically called 'cloud'. Needs thermal band 6
#def scoreClouds(image):
#    return ee.Algorithms.Landsat.simpleCloudScore(image)
    
## Function to mask images using cloud score threshold (lte) 
#def maskClouds(image):
#    mask = image.select(['cloud']).lte(20)
#    return image.updateMask(mask)

#def getImageSum(image):
#    return image.reduceRegion(ee.Reducer.sum())

## from Noel to remove scenes with high cloud score, filter:
## https://groups.google.com/forum/#!searchin/google-earth-engine-developers/simpleCloudScore$20landsat$205/google-earth-engine-developers/9UxD8QR4AV8/BxpNXQgZ8gkJ
def cloud_score (refl_toa):
    cloud_mask = ee.Algorithms.Landsat.simpleCloudScore(refl_toa).select("cloud")
    ccover = cloud_mask.reduceRegion(ee.Reducer.mean(),refl_toa.geometry(),30,None,None,True)
    return refl_toa.set('cloud', ccover)

#collection = collection.map(cloud_score)
#collection = collection.filter(ee.Filter.lt('cloud', 25));

preDam_scored = preDam_clipped.map(cloud_score)
preDam_filtered = preDam_scored.filter(ee.Filter.lt('cloud', 25))

type(preDam_filtered)

widget2 = GoogleMapsView(lng=107.9337, lat=11.3254, zoom= 10, height='400px', width='800px')
widget2.addEarthEngineLayer(preDam_filtered, vis_params={'min': 0, 'max': 0.3, 'bands': 'B3,B2,B1'})
display(widget2)

#postDam_scored = postDam_clipped.map(scoreClouds)

#preDam_imageSum = preDam_scored.map(getImageSum)
#postDam_imageSum = postDam_scored.map(getImageSum)


# Image(url=scored.getThumbUrl({'min':0, 'max':0.3,'bands':'B4,B3,B2', "region": bufferPoly[0]["geometry"]}))
# geojson_map.addEarthEngineLayer(masked, vis_params={'min': 0, 'max': 0.3, 'bands': 'B4,B3,B2'})
# display(geojson_map)

ee.imagecollection.ImageCollection

# Other helpful functions

In [ ]:
# to get a description of the methods available for an object use
help(widget)
# To get the type of object:
type(preDam)